In [1]:
!pip install optimum[onnxruntime-gpu]


In [2]:
!pip install -q transformers

In [3]:
import os
import time
from transformers import AutoModelForImageClassification, ViTImageProcessor
import torch
from PIL import Image
from sklearn.metrics import accuracy_score
import optimum
from optimum.onnxruntime import ORTModelForImageClassification

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/my_model.zip' -d '/content/weights'

In [ ]:
!unzip '/content/drive/MyDrive/data.zip' -d '/content'

In [4]:
path_to_model = "weights/my_model"

# Перемещаем модель на GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

extractor = ViTImageProcessor.from_pretrained(path_to_model)
vit_model = AutoModelForImageClassification.from_pretrained(path_to_model).to(device)

In [5]:
save_directory = "vit_onnx_optimum"

# Указываем, что нужно использовать GPU
ort_model = ORTModelForImageClassification.from_pretrained(path_to_model, export=True,   provider="CUDAExecutionProvider")
ort_model.save_pretrained(save_directory)
extractor.save_pretrained(save_directory)

Framework not specified. Using pt to export to ONNX.
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu118
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/modeling_vit.py:170: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/modeling_vit.py:176: TracerWarning: Converting a tensor to a Python boolean might cause the trace to b

['vit_onnx_optimum/preprocessor_config.json']

In [6]:
extractor_onnx = ViTImageProcessor.from_pretrained(save_directory)
vit_model_onnx = ORTModelForImageClassification.from_pretrained(save_directory)

In [7]:
def model_use(model, img):
    with torch.no_grad():
        logits = model(**img).logits

    predicted_label = logits.argmax(-1).item()

    return model.config.id2label[predicted_label]

In [8]:
path = "data/"
images_list = os.listdir(path)

In [9]:
def size_measurement(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / (1024 ** 2)
    print('model size: {:.3f}MB'.format(size_all_mb))

In [10]:
# Найдем исходный размер модели.
size_measurement(vit_model)

model size: 327.302MB


In [11]:
# Проверим размер ONNX модели.
onnx_file_size = os.path.getsize(save_directory + "/" + "model.onnx")/(1024**2)

print("Размер модели после конвертации в ONNX стал ", onnx_file_size, " мегабайт.")

Размер модели после конвертации в ONNX стал  327.55269622802734  мегабайт.


In [12]:
from tqdm import tqdm
start_time = time.time()

# Собака 1, кошка 0.
target_list = []
predict_list = []

for element in tqdm(images_list):

    image = Image.open(path + element, mode='r', formats=None)

    inputs = extractor_onnx(image, return_tensors="pt").to(device)  # Перемещаем тензоры на GPU
    predict = model_use(vit_model_onnx, inputs)
    target = element[:element.find(".")]

    if target == "dog":
        label = 1
    else:
        label = 0

    target_list.append(label)

    if predict == "dogs":
        pr = 1
    else:
        pr = 0

    predict_list.append(pr)

end_time = time.time()

acc = accuracy_score(target_list, predict_list)
print("Точность модели сконвертированной в ONNX = ", acc)
print("Время обработки изображений модели сконвертированной в ONNX = ", end_time-start_time, " секунд")
print("Скорость обработки изображений у модели сконвертированной в ONNX составила  ", len(images_list)/(end_time-start_time), " картинок в секунду")

100%|██████████| 160/160 [01:36<00:00,  1.65it/s]

Точность модели сконвертированной в ONNX =  0.9875
Время обработки изображений модели сконвертированной в ONNX =  96.93501472473145  секунд
Скорость обработки изображений у модели сконвертированной в ONNX составила   1.6505903512198932  картинок в секунду


## Исходная VIT

In [13]:
def model_use(model, img):
    with torch.no_grad():
        logits = model(**img).logits

    predicted_label = logits.argmax(-1).item()

    return model.config.id2label[predicted_label]

In [16]:
from transformers import AutoFeatureExtractor


In [17]:
path_to_model = "weights/my_model"

extractor = AutoFeatureExtractor.from_pretrained(path_to_model)
vit_model = AutoModelForImageClassification.from_pretrained(path_to_model).to(device)


path = "data/"
images_list = os.listdir(path)
def size_measurement(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / (1024 ** 2)
    print('model size: {:.3f}MB'.format(size_all_mb))
# Найдем исходный размер модели.
size_measurement(vit_model)





/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


model size: 327.302MB


In [18]:
start_time = time.time()

# Собака 1, кошка 0.
target_list = []
predict_list = []

for element in images_list:

    image = Image.open(path + element, mode='r', formats=None)

    inputs = extractor(image, return_tensors="pt").to(device)
    predict = model_use(vit_model, inputs)
    target = element[:element.find(".")]

    if target == "dog":
        label = 1
    else:
        label = 0

    target_list.append(label)

    if predict == "dogs":
        pr = 1
    else:
        pr = 0

    predict_list.append(pr)

end_time = time.time()

In [19]:
acc = accuracy_score(target_list, predict_list)
print("Точность исходной модели= ", acc)
print("Время обработки изображений исходной модели= ", end_time-start_time, " секунд")
print("Скорость обработки изображений у исходной модели составила  ", len(images_list)/(end_time-start_time), " картинок в секунду")


Точность исходной модели=  0.9875
Время обработки изображений исходной модели=  10.828704357147217  секунд
Скорость обработки изображений у исходной модели составила   14.77554421313534  картинок в секунду
